In [186]:
import numpy as np
import pandas as pd
import pdb

In [8]:
pwd

'/Users/mingyuanma/Desktop/hai-cooking/analysis new'

In [9]:
s1 = pd.read_csv("../data/Archive/combined_phase2_s1.csv")
t1 = pd.read_csv("../data/Archive/combined_phase2_t1.csv")



In [10]:
import itertools
from tqdm import tqdm

def find_string_combinations(arr):
    combinations = list(itertools.combinations(arr, 2))
    return combinations

arr = ["chef_chop", "chef_cook", "chef_plate", 
       "souchef_chop", "souchef_cook", "souchef_plate",
      "server_chop", "server_cook", "server_plate"]

In [198]:
def frequency_dictionary(df, spans=3):
    worker_id = [1,2,3]
    tasks_id = [1,2,3]
    steps = spans - 1

    df = df.join(pd.get_dummies(df["taskId"]))
    df = df.rename(columns={0:"idle_2", 1:"chop", 2:"cook", 3:"plate"})

    df = df.join(pd.get_dummies(df["workerId"]))
    warmup = df[df["round"] <= 2]
    warmup = warmup.rename(columns={0:"idle", 1:"chef", 2:"souchef", 3:"server"})
    start = df[df["round"] >= 3]
    start = start.rename(columns={0:"idle", 1:"chef", 2:"server", 3:"souchef"})
    start = start.iloc[:, [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,16,15]]
    df = pd.concat([warmup, start],axis=0)

    df["chef_chop"] = df["chef"] & df["chop"]
    df["chef_cook"] = df["chef"] & df["cook"]
    df["chef_plate"] = df["chef"] & df["plate"]

    df["souchef_chop"] = df["souchef"] & df["chop"]
    df["souchef_cook"] = df["souchef"] & df["cook"]
    df["souchef_plate"] = df["souchef"] & df["plate"]

    df["server_chop"] = df["server"] & df["chop"]
    df["server_cook"] = df["server"] & df["cook"]
    df["server_plate"] = df["server"] & df["plate"]

    arr = ["chef_chop", "chef_cook", "chef_plate", 
           "souchef_chop", "souchef_cook", "souchef_plate",
          "server_chop", "server_cook", "server_plate", "idle"]

    storage_final = {}
    dat = df.copy()
    for i in tqdm(np.arange(1,7)):
        df = df[df["round"] == i]
        feature_panels = find_string_combinations(arr)
        storage = {}
        for responser in df["ResponseId"].unique():
            sub_df = df[df["ResponseId"] == responser]
            for t in np.sort(sub_df["tick"].unique())[:-steps]:
                sub_df = sub_df[(sub_df["tick"] >= t) & (sub_df["tick"] <= t+steps)]
                for feature_panel in feature_panels:
                    feature_a = feature_panel[0]
                    feature_b = feature_panel[1]
                    if np.sum(sub_df[feature_a]) > 0 and np.sum(sub_df[feature_b]) > 0:
                        feature = feature_a + "_" + feature_b
                        count = min(np.sum(sub_df[feature_a]), np.sum(sub_df[feature_b]))
                        if not storage.get(feature):
                            storage[feature] = np.array([count])
                        else: 
                            storage[feature] += count
        for k, v in storage.items():
            storage[k] = np.round(v / len(df["ResponseId"].unique()), 2)
        df = dat
        storage_final[i] = storage
    return storage_final      

In [199]:
s1_frequency = frequency_dictionary(s1, 6)
t1_frequency = frequency_dictionary(t1, 6)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:36<00:00,  6.16s/it]


In [200]:
def dic_to_df(dic):
    round1 = pd.DataFrame(dic[1]).T.reset_index()
    round2 = pd.DataFrame(dic[2]).T.rename(columns={0:1}).reset_index()
    df = pd.merge(round1,round2, how='outer')

    round3 = pd.DataFrame(dic[3]).T.rename(columns={0:2}).reset_index()
    df = pd.merge(df,round3, how='outer')

    round4 = pd.DataFrame(dic[4]).T.rename(columns={0:3}).reset_index()
    df = pd.merge(df,round4, how='outer')

    round5 = pd.DataFrame(dic[5]).T.rename(columns={0:4}).reset_index()
    df = pd.merge(df,round5, how='outer')

    round6 = pd.DataFrame(dic[6]).T.rename(columns={0:5}).reset_index()
    df = pd.merge(df,round6, how='outer')

    df = df.fillna(0).rename(columns = {"index":"strategy", 
                                  0:"round 1",
                                  1:"round 2",
                                  2:"round 3",
                                  3:"round 4",
                                  4:"round 5",
                                  5:"round 6"})
    return df

In [201]:
final = pd.merge(dic_to_df(s1_frequency), dic_to_df(t1_frequency), on="strategy")
s1_final = final.iloc[:, 0:7].rename(columns={
    "round 1_x":"round 1","round 2_x":"round 2","round 3_x":"round 3",
    "round 4_x":"round 4","round 5_x":"round 5","round 6_x":"round 6",
})
t1_final = final.iloc[:, [0,7,8,9,10,11,12]].rename(columns={
    "round 1_y":"round 1","round 2_y":"round 2","round 3_y":"round 3",
    "round 4_y":"round 4","round 5_y":"round 5","round 6_y":"round 6",
})
dif = s1_final.iloc[:,1:] - t1_final.iloc[:,1:]
dif

,round 1,round 2,round 3,round 4,round 5,round 6
0,0.02,-0.04,0.08,-0.10,0.21,0.22
1,-0.03,0.02,0.00,0.00,0.00,0.00
2,-0.18,-0.03,0.00,0.00,0.00,0.00
3,0.03,0.07,0.04,-0.10,0.15,0.15
4,-0.23,-0.18,-0.10,-0.39,-0.75,-0.63
5,0.06,-0.05,0.00,0.00,0.00,0.00
6,0.01,0.25,0.00,0.00,0.00,0.00
7,-0.02,0.13,0.23,0.36,1.03,0.99
8,0.14,-0.37,0.09,0.06,0.88,0.93
9,-0.05,0.13,0.00,0.00,0.00,0.00


In [202]:
dif.iloc[[7,8,4,14,15,16,20],:]

,round 1,round 2,round 3,round 4,round 5,round 6
7,-0.02,0.13,0.23,0.36,1.03,0.99
8,0.14,-0.37,0.09,0.06,0.88,0.93
4,-0.23,-0.18,-0.10,-0.39,-0.75,-0.63
14,0.01,0.14,0.37,0.49,1.35,1.15
15,0.01,-0.01,-0.16,-0.36,-0.72,-0.86
16,0.01,0.06,-0.06,-0.10,-0.34,-0.34
20,0.01,0.15,-0.37,-0.57,-1.09,-1.33


In [203]:
s1_final.iloc[[7,8,4,14,15,16,20],:]

,strategy,round 1,round 2,round 3,round 4,round 5,round 6
7,chef_cook_server_chop,0.98,0.81,3.56,2.88,2.97,3.01
8,chef_cook_idle,4.91,4.76,4.71,4.21,4.31,4.36
4,chef_chop_idle,1.14,1.45,3.13,2.89,2.79,2.70
14,server_chop_idle,1.02,0.84,5.00,4.04,4.26,4.16
15,chef_chop_server_cook,0.48,0.22,0.98,1.44,1.43,1.30
16,chef_cook_server_cook,1.45,0.57,0.80,1.58,1.38,1.38
20,server_cook_idle,2.30,0.96,1.60,2.73,2.71,2.58


In [204]:
t1_final.iloc[[7,8,4,14,15,16,20],:]

,strategy,round 1,round 2,round 3,round 4,round 5,round 6
7,chef_cook_server_chop,1.00,0.68,3.33,2.52,1.94,2.02
8,chef_cook_idle,4.77,5.13,4.62,4.15,3.43,3.43
4,chef_chop_idle,1.37,1.63,3.23,3.28,3.54,3.33
14,server_chop_idle,1.01,0.70,4.63,3.55,2.91,3.01
15,chef_chop_server_cook,0.47,0.23,1.14,1.80,2.15,2.16
16,chef_cook_server_cook,1.44,0.51,0.86,1.68,1.72,1.72
20,server_cook_idle,2.29,0.81,1.97,3.30,3.80,3.91
